# Investigate Net Energy Production
## Load packages and data

In [2]:
import DataRetriever as dr
import pandas as pd
import plotly.express as px

retriever = dr.DataRetriever()

year_2 = retriever.get_data("All-Subsystems-minute-Year2.pkl")

# Add timestamp converted to UTC as index to year_2
year_2["Timestamp UTC"] = pd.to_datetime(year_2["Timestamp"], utc=True, infer_datetime_format=True)
year_2.set_index("Timestamp UTC", inplace=True)

# Load the metadata
metadata = retriever.get_data("metadata-year2.pkl")
metadata.rename(columns={"Unnamed: 0": "Attribute"}, inplace=True)

## Determine which columns are generating and which are consuming
### Generating Attributes

In [2]:
watt_attributes = metadata[metadata["Units"] == "W"]

pv_attributes = [description.startswith("Instantaneous power produced") for description in watt_attributes["Description"]]
gen_attributes = watt_attributes[pv_attributes]
gen_attributes = gen_attributes["Attribute"].tolist()
len(gen_attributes)

2

### Consuming Attributes

In [3]:
loads_attributes = [subsystem=="Loads" for subsystem in watt_attributes["Subsystem"]]
con_attributes = watt_attributes[loads_attributes]
con_attributes = con_attributes["Attribute"].tolist()
len(con_attributes)

24

# Create DataFrame containing only Generating and Consuming attributes

In [4]:
energy_attributes = ["Timestamp"] + gen_attributes + con_attributes

energy_data = year_2[energy_attributes]
energy_data

,Timestamp,PV_Watts3PhTotalW3PhT1,PV_Watts3PhTotalW3PhT2,Load_LRPlugLoadsPowerUsage,Load_KPlugLoadsPowerUsage,Load_MBRPlugLoadsPowerUsage,Load_BR2PlugLoadsPowerUsage,Load_BR3PlugLoadsPowerUsage,Load_BR4PlugLoadsPowerUsage,Load_1stFloorSensHeatPowerUsage,...,Load_RefrigeratorPowerWithStandby,Load_MicrowavePowerWithStandby,Elec_PowerParentADownstairs,Elec_PowerChildBDownstairs,Elec_PowerChildADownstairs,Elec_PowerParentAUpstairs,Elec_PowerChildAUpstairs,Elec_PowerChildBUpstairs,Elec_PowerParentBUpstairs,Elec_PowerParentBDownstairs
Timestamp UTC,,,,,,,,,,,,,,,,,,,,,
2015-02-01 05:01:50+00:00,2015-02-01 00:01:50-05:00,9.541018,8.991452,0.0,0.82,0.000000,0.000000,0.0,0.000000,0.0,...,6.1,1.32,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2015-02-01 05:02:50+00:00,2015-02-01 00:02:50-05:00,9.401856,8.890989,0.0,0.82,56.608574,22.497041,0.0,48.833151,0.0,...,6.1,1.32,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2015-02-01 05:03:50+00:00,2015-02-01 00:03:50-05:00,9.581933,8.956793,0.0,0.82,56.692626,22.480699,0.0,48.967466,0.0,...,6.1,1.32,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2015-02-01 05:04:50+00:00,2015-02-01 00:04:50-05:00,9.571916,9.004457,0.0,0.82,56.662501,22.548397,0.0,48.943777,0.0,...,6.1,1.32,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2015-02-01 05:05:50+00:00,2015-02-01 00:05:50-05:00,9.631621,8.914098,0.0,0.82,56.848702,22.561902,0.0,49.003380,0.0,...,6.1,1.32,0.0,0.0,0.0,73.968090,75.683292,75.476479,71.919773,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-01 04:54:24+00:00,2016-01-31 23:54:24-05:00,9.445323,8.850496,0.0,0.82,77.291357,21.919070,0.0,47.773333,0.0,...,6.1,1.32,0.0,0.0,0.0,74.708458,72.596176,70.134552,72.768318,0.0
2016-02-01 04:55:24+00:00,2016-01-31 23:55:24-05:00,9.485347,8.884768,0.0,0.82,77.387500,21.970587,0.0,47.712440,0.0,...,6.1,1.32,0.0,0.0,0.0,74.349522,72.518196,70.231998,72.466225,0.0
2016-02-01 04:56:24+00:00,2016-01-31 23:56:24-05:00,9.432410,8.789854,0.0,0.82,77.447937,21.978156,0.0,47.817486,0.0,...,6.1,1.32,0.0,0.0,0.0,74.579205,72.725151,70.093410,72.705517,0.0


### Create new attributes "Generated", "Consumed", and "Net Energy"

In [21]:
total_generated = energy_data[gen_attributes].sum().sum()/1000
total_consumed = energy_data[con_attributes].sum().sum()/1000
print(total_generated, total_consumed, total_generated - total_consumed)

835493.003019695 441293.15732943726 394199.8456902578


## Add sum of generating and consuming attributes to energy_data  DataFrame

In [25]:
energy_data["Sum Generating"] = energy_data[gen_attributes].sum(axis=1)
energy_data["Sum Consuming"] = energy_data[con_attributes].sum(axis=1)
energy_data["Net Energy"] = energy_data["Sum Generating"] - energy_data["Sum Consuming"]
energy_data["Turned On"] = energy_data["Net Energy"] > 0

C:\Users\madsc\AppData\Local\Temp\ipykernel_21044\3496694228.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\madsc\AppData\Local\Temp\ipykernel_21044\3496694228.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\madsc\AppData\Local\Temp\ipykernel_21044\3496694228.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

# Plotting consuming attributes

In [ ]:
fig = px.bar(data_frame=energy_data,
             x="Timestamp",
             y="Sum Consuming")
fig.update_xaxes(range=["2015-03-13", "2015-03-23"])
fig.show()

## Plotting generating attributes

In [ ]:
fig = px.bar(data_frame=energy_data,
             x="Timestamp",
             y="Sum Generating")
fig.update_xaxes(range=["2015-03-13", "2015-03-23"])
fig.show()

## Plotting Net Energy

In [ ]:
fig = px.bar(data_frame=energy_data,
             x="Timestamp",
             y="Net Energy",
             color="Turned On",
             color_discrete_sequence=["blue", "black"])
fig.update_xaxes(range=["2015-03-13", "2015-03-23"])
fig.show()